In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib
from tqdm.notebook import tqdm
import time
from datetime import datetime

# 1 Часть - основные города

In [2]:
def checkEnd(url):
    the_string = url.rsplit('php/',1)[1]
    srch = re.search('[а-яА-Я]', the_string) 
    if srch == None:
        str_url = urllib.parse.unquote(the_string)
    else:
        str_url = the_string
    #
    return str_url

In [3]:
def start(url):
    str_url = checkEnd(url)
    lst_splt = str_url.split("_")

    nmbr = lst_splt[-1]
    # 1 - тип транспорта
    tp_ts = lst_splt[-2]

    lst_type_ok = ['автобус','трамвай','троллейбус']

    i=0
    while not (any(map(str.isdigit, nmbr)) or ("б/н" in nmbr)):
        try:
            nmbr = lst_splt[i]
            if (any(map(str.isdigit, nmbr)) or ("б/н" in nmbr)):
                break
            i+=1
        except:
            nmbr = 0
            break
    #
    try:
        nmbr = nmbr.replace("-", "_")
    except:
        pass

    i=0
    while not tp_ts in lst_type_ok:
        try:
            tp_ts = lst_splt[i]
            if tp_ts in lst_type_ok:
                break
            i+=1
        except:
            tp_ts = "автобус другой"
            break

    # 
    return nmbr,tp_ts

In [4]:
def getTable(table_rows):
    l = []
    for tr in table_rows:
        try:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            l.append(row)
        except:
            pass
    #
    return l

In [5]:
def end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url):
    lst_all = [city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw]
    
    def foo(nlst,i):
        return nlst[i][1]

    for i in range(9):
        try: 
            tmp = foo(nlst,i)
        except:
            tmp = None
        lst_all.append(tmp)
    # 
    lst_all.append(url)
    return lst_all

In [6]:
def getInfo(url,region,city):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    nmbr, tp_ts = start(url)
    
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    #####

    all_tables = bs.find_all("table")

    #######
    l2,l3,l4,l5,l6,l7,l8 = [],[],[],[],[],[],[]
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if 'Содержание'in l[0][0]:
            l2 = l
        elif ('направление'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if ('прям' in l[0][0]) | ('прям' in tmp):
                if '№п' not in tmp:
                    l3 = l
                else:
                    l5 = l
            elif ('обратн' in l[0][0]) | ('обратн' in tmp):
                if '№п' in tmp:
                    l6 = l
            #
        elif ('обратное'in l[0][0]) & (len(l[0][0]) < 40):
            try: 
                tmp = l[1][0]
            except:
                tmp = ''
            if '№п' in tmp:
                l6 = l
        elif 'работы'in l[0][0]:
            l7 = l
        else:
            l8 = l
    # 

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")
    rt_nm = str1.split("\n")[1][2:].split("маршрут ")[1]
    rt_nm = rt_nm.split('"', 1)[1][:-1]

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1]
    try:
        route_bckw = l3[1][1]
    except:
        route_bckw = 'кольцевой'
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    try:
        stops_bckw = stops(l6)
    except:
        stops_bckw = 'кольцевой'
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 
    
    work_time,interv,vpsk,rt_time,length,orgnztr,podvst,money,plc_rst = None,None,None,None,None,None,None,None,None

    i=0
    for i in range(len(nlst)):
        if (("ремя работы" in nlst[i][0]) | ("ежим работы" in nlst[i][0])):
            work_time = nlst[i][1]
        elif "нтервал" in nlst[i][0]:
            interv = nlst[i][1]
        elif "ыпуск" in nlst[i][0]:
            vpsk = nlst[i][1]
        elif "ремя в пути" in nlst[i][0]:
            rt_time = nlst[i][1]
        elif ("ротяжённость" in nlst[i][0]) | ("ротяженность" in nlst[i][0]):
            length = nlst[i][1]
        elif "бслуживающие" in nlst[i][0]:
            orgnztr = nlst[i][1]
        elif "одвижной" in nlst[i][0]:
            podvst = nlst[i][1]
        elif "тоимость проезда" in nlst[i][0]:
            if money == None:
                money = nlst[i][1]
        elif ("есто" in nlst[i][0]) & ("отсто" in nlst[i][0]):
            plc_rst = nlst[i][1]
    # 
    
    tmp_lst = [work_time,interv,vpsk,rt_time,length,orgnztr,podvst,money,plc_rst]

    nlst2=[]
    for i in range(len(tmp_lst)):
        tl = [0,tmp_lst[i]]
        nlst2.append(tl)
    # 
    nlst = nlst2
    
    lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)

    # 
    return lst_all

In [22]:
def getInfo2(url,region,city):
    # Сызрань спешл
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')
    
    nmbr, tp_ts = start(url)
    
    all_tables = bs.find_all("table")
    
        # 3 - наименование маршрута
    rt_nm = str(bs.find_all('b')[2]).replace("<b>","").replace("</b>","")

    def getLines(lst_s,nmb):
        lst_ok = []

        for i in range(len(lst_s)):
            lst1 = lst_s[i][nmb].split("\n")
            for s in lst1:
                if s != '':
                    lst_ok.append(s)
        # 
        return lst_ok

    def stops(lst_s,nmb):
        lst_ok = getLines(lst_s,nmb)

        str1=''
        for row in lst_ok:
            if row != '\n':
                row = row.replace("\n", "")
                str1 = str1 + "_" + row
        # 
        str1 = str1[1:]
        return str1

    ####
    
    i=0
    for i in range(len(all_tables)):
        l = getTable(all_tables[i])
        if "в сторону" in l[0][0]:
            l5 = getTable(all_tables[i+2])
            l6 = getTable(all_tables[i+3])
            break
        else:
            l5 = getTable(all_tables[5])
            l6 = getTable(all_tables[6])
    # 

    ####
    # 4 -маршрут следования

    route_frw = stops(l5,0)
    try:
        route_bckw = stops(l6,0)
    except:
        route_bckw = 'кольцевой'
    ####

    # 5,6 - список оп
    stops_frw = stops(l5,1)
    try:
        stops_bckw = stops(l6,1)
    except:
        stops_bckw = 'кольцевой'
    ####
    
    # nlst = []

    lst_fnd = bs.find_all(["ul","p"])
    orgnztr,podv_sost,plc_rst,money = '','','',''
    i=0
    for i in range(len(lst_fnd)):
        if "бслуживающ" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                orgnztr = str(orgnztr) + "_" + lst_fnd[i].text
                i+=1
        elif "одвижной состав" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                podv_sost = str(podv_sost) + "_" + lst_fnd[i].text
                i+=1
        elif "тоимость проезд" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                money = str(money) + "_" + lst_fnd[i].text
                i+=1
        elif "Обед" in lst_fnd[i].text:
            i+=1
            while lst_fnd[i].name == 'p':
                plc_rst = str(plc_rst) + "_" + lst_fnd[i].text
                i+=1
    # 
    orgnztr = orgnztr.replace("\n","").replace("None_","").replace("_","",1)
    podv_sost = podv_sost.replace("\n","").replace("None_","").replace("_","",1)
    plc_rst = plc_rst.replace("\n","").replace("None_","").replace("_","",1)
    money = money.replace("\n","").replace("None_","").replace("_","",1)

    nlst = [[0,None],[0,None],[0,None],[0,None],[0,None],[0,orgnztr],[0,podv_sost],[0,money],[0,plc_rst]]


    lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)
    
    return lst_all

In [8]:
def checkGrpNm(tmp_grp_nm):
    if (tmp_grp_nm == 'маршруты') | ("городск" in tmp_grp_nm):
        grp_name = 'муниципальный'
    elif 'коммерческ' in tmp_grp_nm:
        grp_name = 'коммерческий'
    elif  'пригородн' in tmp_grp_nm:
        grp_name = 'пригородный'
    elif ("междугород" in tmp_grp_nm) | ('межмуницип' in tmp_grp_nm):
        grp_name = 'межмуниципальный'
    elif "сезон" in tmp_grp_nm:
        grp_name = 'сезонный'
    elif "специальн" in tmp_grp_nm:
        grp_name = 'специальный'
    else:
        grp_name = tmp_grp_nm
    # 
    return grp_name

In [9]:
# 
def linksCity(city):

    url_grp = 'http://www.samaratrans.info/wiki/index.php/' + city
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('a')
    links=[]
    for link in stfa:
        try:
            if 'маршрут' in link.get("title"):
                lsh = 'http://www.samaratrans.info' + link.get("href")
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [10]:
def routeLinks(url_grp):
    links=[]
    req = requests.get(url_grp)
    bs = BeautifulSoup(req.text, 'html.parser')
    stfa = bs.find_all('table')
    
    cnt=2
    for i in range(len(stfa)):
        table_rows = stfa[i]
        l = getTable(table_rows)
        if 'Наименование маршрута\n' in l[0]:
            cnt = i
            break
    # 
    lst_find = stfa[cnt].find_all('a')
    for link in lst_find:
        try:
            lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
            lnk_hr = link.get("href")
            ch = checkEnd(lnk_hr)
            if not any((c in ch) for c in lst_notok):
                lsh = 'http://www.samaratrans.info' + lnk_hr
                if lsh not in links:
                    links.append(lsh)
        except:
            pass
    # 
    return links

In [11]:
def getCityData(city):
    lst_links = []
    lst_links = linksCity(city)
    all_routes=[]
    for grp in range(len(lst_links)):
        tmp_grp_nm = checkEnd(lst_links[grp]).split("_",2)[-1]
        grp_name = checkGrpNm(tmp_grp_nm)
        
        try:
            lst_group = []
            lst_group = routeLinks(lst_links[grp])
        except:
            lst_group = []
        for rtl in range(len(lst_group)):
            try:
                tmp_lnk_nm = checkEnd(lst_group[rtl])
                lst_notok = ['расписан', 'парк', '.gif', '.png', 'перевозчик', 'edit', 'ТД', 'ТТУ']
                if not any((c in tmp_lnk_nm) for c in lst_notok):
                    try:
                        one_route = getInfo(lst_group[rtl],grp_name,city)
                    except:
                        one_route = getInfo2(lst_group[rtl],grp_name,city)
                    all_routes.append(one_route)
                #
            except:
                one_route = [lst_group[rtl]] * 19
                if one_route not in all_routes:
                    all_routes.append(one_route)
                #
    return all_routes
# 

In [12]:
def mainCities():
    lst_ct = ['Самара', 'Тольятти', 'Сызрань', 'Новокуйбышевск', 
          'Чапаевск', 'Жигулевск', 'Отрадный', 'Кинель', 'Похвистнево', 'Безенчук']
    lst_big = []
    for i in tqdm(range(len(lst_ct))):
        city = lst_ct[i]
        lst = getCityData(city)
        lst_big = lst_big + lst
    # 
    return lst_big

# 2 Часть - другие города (ссылка ниже)

In [13]:
def getOtherLinks(url_ot):
    req = requests.get(url_ot)
    bs = BeautifulSoup(req.text, 'html.parser')

    all_lnks = bs.find_all('a')
    lst_rasp=[]
    i=0
    for link in all_lnks:
        if "расписани" in str(link.get('title')):
            lsh = "http://www.samaratrans.info" + link.get('href')
            lst_rasp.append(lsh)
    # 

    return lst_rasp

In [14]:
def getInfo3(url):

    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')

    city = checkEnd(url).split("_",1)[0]

    ####
    # get stops
    # stops_frw

    all_p = bs.find_all('p')
    p=0
    lst_txt = []
    for p in range(len(all_p)):
        if "Промежуточные" in str(all_p[p]):
            lst_txt.append(all_p[p].text)
    # 
    ####
    # костыль!
    if url == "http://www.samaratrans.info/wiki/index.php/%D0%A7%D0%B5%D0%BB%D0%BD%D0%BE-%D0%92%D0%B5%D1%80%D1%88%D0%B8%D0%BD%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F":
        str_bad = "Промежуточные остановки: нет\n"
        lst_txt.remove(str_bad)
    # 

    ####
    
    i=0
    lst_stops=[]
    for i in range(len(lst_txt)):
        txt = lst_txt[i].split(":",1)[1].replace(", ","_").replace("\n","")
        if "остановки:" in lst_txt[i]:
            lst_stops.append(txt)
        else:
            txt2 = ''
            while "остановки:" not in lst_txt[i]:
                ii = i
                txt = lst_txt[i].replace(", ","_").replace("\n","").replace("Промежуточные остановки ","")
                txt2 = txt2 + "[!!!]" + txt
                i+=1
                if "остановки:" in lst_txt[i]:
                    if len(lst_stops) == 0:
                        lst_stops.append(txt2)
                        break
                    elif "[!!!]" not in lst_stops[-1]:
                        lst_stops.append(txt2)
                        break
    # 
    ####
    ####
    i=0
    all_tables = bs.find_all('table')
    for i in range(len(all_tables)):
        l = getTable(bs.find_all('table')[0])
        if "Расписание" in l[0][0]:
            l1 = l
            break
        else:
            l1 = getTable(bs.find_all('table')[0])
    # 
    lst_nms = l1[0][0].split("\n")
    ####

    ####
    lst_reg = []
    lst_tpts = []
    lst_rtnbr = []
    lst_rtnm = []
    for i in range(len(lst_nms)):
        str1 = lst_nms[i]
        if "Расписание" in str1:
            if (" муниципальн" in str1) | (" городск" in str1):
                region = "муниципальный"
            elif (" межмуниципальн" in str1) | (" межрегион" in str1) | (" пригород" in str1):
                region = "межмуниципальный"
            else:
                region = "другой"
        else:
            if "маршрут" in str1:
    #             lst_rt.append(str1)
                tp_ts, rt = str1.split(" ",1)[1].split("маршрут ",1)
                rt_nbr, rt_nm = rt.split('"',1)
                rt_nbr = rt_nbr.replace(" ", "").replace("№", "").replace("-", "_").replace("/", "_")
                rt_nm = rt_nm[:-1]

                if "втобус" in tp_ts:
                    tp_ts = "автобус"
                elif "рамвай" in tp_ts:
                    tp_ts = "трамвай"
                elif "роллейбус" in tp_ts:
                    tp_ts = "троллейбус"
                else:
                    tp_ts = "другой"
                # 
                lst_rtnbr.append(rt_nbr)
                lst_rtnm.append(rt_nm)
                lst_tpts.append(tp_ts)

                lst_reg.append(region)
    # 
    ####

    ####
    lst_all = []
    def foo(lst,nmbr):
        try:
            answ = lst[nmbr]
        except:
            answ = None
        return answ
    i=0
    for i in range(len(lst_stops)):
        tp_ts = lst_tpts[i]
        region = lst_reg[i]
        nmbr = lst_rtnbr[i]
        rt_nm = lst_rtnm[i]
        route_frw = None
        route_bckw = None
        stops_frw = lst_stops[i]
        stops_bckw = None
        nlst = []
        small_lst_all = end(city,tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,nlst,url)
        lst_all.append(small_lst_all)
    # 
    ####
#     ####

    return lst_all

In [15]:
def smallCities():

    url_ot = "http://www.samaratrans.info/wiki/index.php/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F"

    lst_rasp = getOtherLinks(url_ot)

    lst_big2 = []
    i=0
    for i in tqdm(range(len(lst_rasp))):
        url = lst_rasp[i]
        try:
            lst = getInfo3(url)
        except:
            lst = [[url] * 19]
        #
        for j in range(len(lst)):
            lst_big2.append(lst[j])
    # 

    return lst_big2

In [16]:
def createReestr():

    lst_big = mainCities()
    lst_big2 = smallCities()

#     clmn = []
#     for i in range(1,len(lst_big[0])+1):
#         str1 = str(i)+'str'
#         clmn.append(str1)
#     # 

    
    clmn = ["населенный_пункт","тип_тс","тип_маршрута","номер_маршрута",
            "наим_маршрута","маршут_след_прям","маршут_след_обрат",
            "список_оп_прям","список_оп_обрат","режим_работы","интервалы","выпуск","время_в_пути",
            "протяженность","предприятие","подв_состав","стоимость_проезда","место_отстоя","ссылка"]
    
    df = pd.DataFrame(data=lst_big,columns=clmn)
    df2 = pd.DataFrame(data=lst_big2,columns=clmn)

    reestr = df.append(df2).reset_index(drop=True)

    return reestr

In [17]:
%time reestr = createReestr()


Wall time: 4min 3s


In [18]:
# можно посмотреть, сколько ошибок (если ошибка - в таблице ссылка на маршрут)
# должно быть 0
lst_notok = list(set(list(reestr[reestr["населенный_пункт"].str.contains("www")]["ссылка"])))


In [19]:
if len(lst_notok) == 0:
    print(len(lst_notok))
    str_date = "{:%d_%H%M}".format(datetime.now())
    reestr.to_csv("reestr_{}.csv".format(str_date),encoding='utf-8-sig',sep=";",index=False)
# 

0
